<a href="https://colab.research.google.com/github/jisang93/SNS_Project/blob/master/Insta_Crwaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import requests, re, json, urllib, datetime
import pandas as pd
from bs4 import BeautifulSoup

#search = '%EC%9D%B4%EC%8A%B9%EA%B8%B0'
#def insta(query):
#search = urllib.parse.quote(query)

# 헤더 저장
headers = {
            'cookie' : 'ig_did=1DB997A1-85F5-410B-A44B-DB3A31DE9BF2; mid=XxLtBgALAAHy9DlenU_IjVp8c-vI; csrftoken=w8fk3ZvDCg3GeMsgOGjj4c7C1J1CDDEv; ds_user_id=2164165712; sessionid=2164165712%3A0i2iNaTHNVLjRS%3A25; shbid=980; shbts=1597821443.3613222; rur=VLL; urlgen="{\"222.107.238.125\": 4766}:1k90Xa:Xh5aOhIn5p2sNhAMjIh4OesdfJk"',
            'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
            }
# 해시 태그 설정
tags = "20대"

res = requests.get('https://www.instagram.com/explore/tags/{}/?__a=1'.format(tags), headers = headers)
res.encoding = 'utf-8'
js = json.loads(res.text)

In [46]:
user_id = []
for i in js['graphql']['hashtag']['edge_hashtag_to_media']['edges']:
    # user 이름으로 접속할 경우 게시글이 다른 게시물로 넘어감
    
    user_id.append('https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B%22id%22%3A%{}%22%2C%22first%22%3A12%7D'.format(i['node']['owner']['id']))
    
user_id[:5]

['https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B%22id%22%3A%21685152986%22%2C%22first%22%3A12%7D',
 'https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B%22id%22%3A%4847179109%22%2C%22first%22%3A12%7D',
 'https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B%22id%22%3A%3618337678%22%2C%22first%22%3A12%7D',
 'https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B%22id%22%3A%2502885322%22%2C%22first%22%3A12%7D',
 'https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B%22id%22%3A%1790527918%22%2C%22first%22%3A12%7D']

In [47]:
user_nm = []
for i in short_code:
    res = requests.get(i, headers=headers)
    res.encoding = 'utf-8'
    print(res)
    js = json.loads(res.text)
js

<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [200]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400

{'message': 'Variables are invalid JSON.', 'status': 'fail'}

In [26]:
url = 'https://www.instagram.com/ji__won124/'
res = requests.post(url, headers=headers)
res

<Response [405]>